# Download some COCO Data

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
%cd 'drive/MyDrive/fiftyone'
%ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/fiftyone
coco-2017/


In [ ]:
%cd '../datasets/detector-recipe'

/content/drive/MyDrive/datasets/detector-recipe


In [ ]:
!pip install fiftyone

In [ ]:
!pip install fiftyone-db-ubuntu2204

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 MB 11.9 MB/s eta 0:00:00


In [ ]:
import fiftyone as fo
import fiftyone.zoo as foz

dataset = foz.load_zoo_dataset(
    "coco-2017",
    splits=["validation","train"],
    classes=["person", "car"],
    dataset_name="detector-recipe",
    max_samples=10000,
)

INFO:fiftyone.zoo.datasets:Downloading split 'validation' to '/root/fiftyone/coco-2017/validation' if necessary


Found annotations at '/root/fiftyone/coco-2017/raw/instances_val2017.json'


INFO:fiftyone.utils.coco:Found annotations at '/root/fiftyone/coco-2017/raw/instances_val2017.json'


Only found 2869 (<10000) samples matching your requirements


Sufficient images already downloaded


...

INFO:fiftyone.utils.coco:Writing annotations for 10000 downloaded samples to '/root/fiftyone/coco-2017/train/labels.json'


Dataset info written to '/root/fiftyone/coco-2017/info.json'


INFO:fiftyone.zoo.datasets:Dataset info written to '/root/fiftyone/coco-2017/info.json'


Loading existing dataset 'detector-recipe'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


INFO:fiftyone.zoo.datasets:Loading existing dataset 'detector-recipe'. To reload from disk, either delete the existing dataset or provide a custom `dataset_name` to use


In [ ]:
dataset

Name:        detector-recipe
Media type:  image
Num samples: 5000
Persistent:  False
Tags:        []
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)

In [ ]:
%ls

coco-2017/  info.json


# Load in the data

In [1]:
from google.colab import drive
drive.mount("/content/drive")
%cd 'drive/MyDrive/datasets/detector-recipe'
%ls

Mounted at /content/drive
/content/drive/MyDrive/datasets/detector-recipe
coco-2017/  info.json


In [2]:
%cd detector-recipe/
%ls

[Errno 2] No such file or directory: 'detector-recipe/'
/content/drive/MyDrive/datasets/detector-recipe
coco-2017/  info.json


In [42]:
import torch
from torchvision.datasets import CocoDetection
from torch.utils.data import DataLoader
from torchvision.transforms import PILToTensor, Resize, Lambda, v2

preproc = v2.Compose([v2.PILToTensor(),
                      v2.Resize((640,640))])


def output_transform(output):
  t = torch.zeros([1,7,100])
  t[:, 4, :] = torch.tensor([1]*100).unsqueeze(dim=0) # initialize everything to background
  for idx, det in enumerate(output):
    if det['category_id'] == 1: #person is 1
      t[:, :, idx] = torch.concat((torch.tensor(det['bbox']), torch.tensor([0,1,0]))).unsqueeze(dim=0)
    if det['category_id'] == 3: #car is 3
      t[:, :, idx] = torch.concat((torch.tensor(det['bbox']), torch.tensor([0,0,1]))).unsqueeze(dim=0)
  return t[0]


dataset = CocoDetection(root='./coco-2017/train/data', annFile='./coco-2017/train/labels.json', transform=preproc, target_transform=output_transform)

loading annotations into memory...
Done (t=2.99s)
creating index...
index created!


In [43]:
train_loader = DataLoader(dataset)

In [ ]:
for data, target in train_loader:
  print(data)
  print(target)
  break

In [45]:
target.shape

torch.Size([1, 7, 100])

In [ ]:
import torch
t = torch.zeros([1,7,100])
t[:, 4, :] = torch.tensor([1]*100).unsqueeze(dim=0)
t[:, :, 1] = torch.concat((torch.tensor(target[1]['bbox']), torch.tensor([0,1,0]))).unsqueeze(0)
t